In [2]:
import numpy as np
from numpy import expand_dims

import tensorflow as tf

In [3]:
trainedModel = tf.keras.models.load_model(
    '../../models/34', 
    custom_objects=None, 
    compile=True, 
    options=None
)

In [4]:
trainedModel.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_9 (Sequential)   (None, 200, 200, 3)       0         
                                                                 
 sequential_10 (Sequential)  (None, 200, 200, 3)       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 198, 198, 32)      896       
                                                                 
 batch_normalization_18 (Bat  (None, 198, 198, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 99, 99, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 97, 97, 64)      

In [9]:
IMAGE_SIZE = 200
BATCH_SIZE = 80

In [10]:
aql_dataset = tf.keras.utils.image_dataset_from_directory(
      "../../data",
      shuffle=True,
      image_size=(IMAGE_SIZE, IMAGE_SIZE),
      batch_size = BATCH_SIZE,
      # labels=None
      )

print(aql_dataset)

Found 615 files belonging to 7 classes.
<BatchDataset element_spec=(TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [ ]:
# len(aql_dataset)

In [11]:
aql_ds = aql_dataset.take(2)
# len(aql_ds)

In [12]:
for element in aql_ds.as_numpy_iterator(): 
    X, y = element
    yhat = trainedModel.predict(X)
    prediction = (np.argmax(yhat, axis=-1))
    # pre.update_state(y, yhat)
    # re.update_state(y, yhat)
    # acc.update_state(y, yhat)
    unique2, counts2 = np.unique(y, return_counts=True)
    dict(zip(unique2, counts2))
    # print (yhat)
    # print (y)

print (prediction)

unique2, counts2 = np.unique(prediction, return_counts=True)
statistics = dict(zip(unique2, counts2))

sampleBatch = (len(prediction))

print (statistics)

[2 2 3 2 0 3 0 2 0 3 3 3 3 2 1 3 2 2 2 1 0 3 2 2 0 3 2 2 2 2 2 2 0 2 2 3 0
 0 0 0 2 2 0 0 3 2 1 3 0 0 0 0 2 1 3 2 0 2 2 0 2 1 1 2 2 0 2 3 1 0 0 0 0 2
 3 1 2 2 0 2]
{0: 24, 1: 8, 2: 33, 3: 15}


In [13]:
# statistics = dict(zip(unique2, counts2))
print (statistics)

{0: 24, 1: 8, 2: 33, 3: 15}


In [14]:
blotchApple = statistics[0]    
appleScore = statistics[1]
rotApple = statistics[2]
scabApple = statistics[3] 
rejectedApple = int(blotchApple) + int(rotApple) + int(scabApple)

healthyPercentage = round(appleScore / (sampleBatch) * 100)
blotchPercentage = round(blotchApple / (sampleBatch) * 100)
rotPercentage = round(rotApple / (sampleBatch) * 100)
scabPercentage = round(scabApple / (sampleBatch) * 100)
rejectedPercentage = round(blotchPercentage + rotPercentage + scabPercentage)

def aqlTestScore(): 
    
    if sampleBatch != 80:
    
        print ("A batch of exactly 80 apples is required for a correct quality control")
        
    else:    
       
        if appleScore >=79:                                          
            status = 'Class 1'
        elif appleScore >=75:
            status = 'Class 2'
        elif appleScore >=73:
            status = 'Class 3'
        else:
            status = 'Rejected'
         
        print(f'The batch has been qualified as: {status}\n')
                
        print (f'The total batch of {sampleBatch} apples consists of:\n'
        f'Healthy apples:    {appleScore}    ({healthyPercentage}%)\n'
        f'Blotched apples:   {blotchApple}    ({blotchPercentage}%)\n'
        f'Rotten apples:     {rotApple}    ({rotPercentage}%)\n'
        f'Scabbed apples:    {scabApple}    ({scabPercentage}%)\n')    
        
        print(f'The total rejected percentage is {rejectedPercentage}%, which is a total amount of', rejectedApple, 'apples.')
        # break    

aqlTestScore()


The batch has been qualified as: Rejected

The total batch of 80 apples consists of:
Healthy apples:    8    (10%)
Blotched apples:   24    (30%)
Rotten apples:     33    (41%)
Scabbed apples:    15    (19%)

The total rejected percentage is 90%, which is a total amount of 72 apples.
